# Martingale Backtesting 

This notebook will look to generate some sort os trading signals and store it in pandas dataframe column. that waill then be used in backtesting.py to backtest using a martingale risk management approach

In [1]:
import MetaTrader5 as mt5
import pandas as pd
import ta
import time
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import cycler
%matplotlib inline

In [4]:
# Settings 
symbol = 'XAUUSD'
timeframe = mt5.TIMEFRAME_H1
start_date = datetime(2023, 1, 1)
end_date = datetime.now()

# Function 
def GetMTData():
    mt5.initialize()
    bars = pd.DataFrame(mt5.copy_rates_range(symbol, timeframe, start_date, end_date))
    bars['time'] = pd.to_datetime(bars['time'], unit='s')
    bars.columns = bars.columns.str.capitalize()
    return bars

df  = GetMTData()

In [6]:
# Function to generate signals based on EMA crossover
def generate_signals(data):
    # Calculate EMAs
    data['EMA20'] = ta.trend.ema_indicator(data['Close'], window=20)
    data['EMA120'] = ta.trend.ema_indicator(data['Close'], window=120)

    
    data['Signals'] = 0
    
    prev_signal = 0

    # Generate signals based on EMA crossover
    for i in range (1, len(data)):
        if data['EMA20'][i]>data['EMA120'][i] and data['EMA20'][i-1] <= data['EMA120'][i-1]:
                data.at[i, 'Signals'] = 2
                prev_signal = 2
        elif data['EMA20'][i]<data['EMA120'][i] and data['EMA20'][i-1] >= data['EMA120'][i-1]:
                data.at[i, 'Signals'] = 1
                prev_signal = 1
        else:
            data.at[i, 'Signals'] = 0
            prev_signal = 0
    return data


df = generate_signals(df)
df

,Time,Open,High,Low,Close,Tick_volume,Spread,Real_volume,EMA20,EMA120,Signals
0,2023-01-03 01:00:00,1826.92,1830.94,1824.21,1829.48,2436,1,0,NaN,NaN,0
1,2023-01-03 02:00:00,1829.48,1832.23,1828.82,1830.66,2324,2,0,NaN,NaN,0
2,2023-01-03 03:00:00,1830.60,1832.46,1826.88,1828.54,5329,0,0,NaN,NaN,0
3,2023-01-03 04:00:00,1828.56,1839.97,1828.31,1838.37,4111,0,0,NaN,NaN,0
4,2023-01-03 05:00:00,1838.37,1843.16,1838.37,1841.26,3628,0,0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
7126,2024-03-15 19:00:00,2160.04,2163.51,2157.97,2158.85,6881,1,0,2164.286920,2161.939753,0
7127,2024-03-15 20:00:00,2158.85,2160.62,2155.21,2156.82,5572,5,0,2163.575784,2161.855129,0
7128,2024-03-15 21:00:00,2156.82,2160.32,2156.59,2159.37,3098,5,0,2163.175234,2161.814053,0
7129,2024-03-15 22:00:00,2159.33,2159.50,2156.41,2156.93,2450,5,0,2162.580449,2161.733324,0
